In [ ]:
import os 
import torch

from diffusers import UNetSpatioTemporalConditionModel, StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video, export_to_gif

In [ ]:
unet = UNetSpatioTemporalConditionModel.from_pretrained(
    "/home/lyq/data/stable-video-diffusion-img2vid-xt",
    subfolder="unet",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False,
)
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "/home/lyq/data/stable-video-diffusion-img2vid-xt",
    unet=unet,
    low_cpu_mem_usage=False,
    torch_dtype=torch.float16, variant="fp16", local_files_only=True,
)
pipe.to("cuda:0")

In [ ]:
image = load_image('/home/lyq/code/time_reversal/test_data/video_frames/dolomite_clip3/frame_00000.jpg')
image = image.resize((1024, 576))

generator = torch.manual_seed(-1)
with torch.inference_mode():
    frames = pipe(image,
                num_frames=24,
                width=1024,
                height=576,
                decode_chunk_size=8, generator=generator, motion_bucket_id=127, fps=8, num_inference_steps=30).frames[0]
# export_to_video(frames, "generated.mp4", fps=7)
imageio.mimsave("generated.mp4", frames, fps=7)